# Test GPU+CUDA

In [1]:
import tensorflow as tf
tf.__version__

'1.8.0'

In [2]:
# a = tf.constant([1.0, 2.0, 3.0 ,4.0, 5.0, 6.0], shape=[2, 3], name='a')
# b = tf.constant([1.0, 2.0, 3.0 ,4.0, 5.0, 6.0], shape=[3, 2], name='b')
# c = tf.matmul(a, b)

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# print(sess.run(c))

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5561530569249808738
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1479989657
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13033214203976661813
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


# Preprocess

+ 手写字体：
    1. 上传自己的手写图片灰阶照片 
    <p align="center">
      <img src="handwriting_preparation\images\yun.jpg" alt="handwritten-pic",  width="600"/>
    </p>
    
    2. 使用Tesseract+jTessBoxEditor获取每个charactor的内容与坐标box坐标信息
    
    <p align="center">
      <img src="README.asserts/char_box.png" alt="char-box",  width="400"/>
    </p>
    
    每个字对应的box信息：
    <p align="center">
      <img src="README.asserts/box.png" alt="box-info",  width="400"/>
    </p>
+ 书法字体（target）：
    
    1. 由于名家书法作品图片不够齐全，所以使用ttf格式的计算机编码字体生成图像
    2. 对比之后，选用“迷你简黄草”字体
    <p align="center">
      <img src="README.asserts/font_viewer.png" alt="char-box",  width="400"/>
    </p>


In [4]:
# !python font2img_finetune.py

    
# 制作数据集

+ 将**手写字体**-**书法字体**pairwise地生成图片，组成数据集
    <p align="center">
      <img src="data\paired_images_finetune\136_0000.jpg" alt="pairwise",  width="200"/>
      <img src="data\paired_images_finetune\136_0001.jpg" alt="pairwise",  width="200"/>
      <img src="data\paired_images_finetune\136_0002.jpg" alt="pairwise",  width="200"/>
      <img src="data\paired_images_finetune\136_0003.jpg" alt="pairwise",  width="200"/>
      <img src="data\paired_images_finetune\136_0004.jpg" alt="pairwise",  width="200"/>
      <img src="data\paired_images_finetune\136_0005.jpg" alt="pairwise",  width="200"/>
      <img src="data\paired_images_finetune\136_0006.jpg" alt="pairwise",  width="200"/>
    </p>
    

# Train

In [5]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import print_function
from model.preprocessing_helper import CANVAS_SIZE, EMBEDDING_DIM
from model.unet import UNet


config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [6]:
################ train-params ###############

experiment_dir='experiments_finetune'    # experiment directory, data, samples,checkpoints,etc
experiment_id=1         # sequence id for the experiments you prepare to run
image_size=CANVAS_SIZE  # size of your input and output image
L1_penalty=100      # weight for L1 loss
Lconst_penalty=15   # weight for const loss
Ltv_penalty=0.0     # weight for tv loss
Lcategory_penalty=1.0  # weight for tv loss
embedding_num=40   # number for distinct embeddings
embedding_dim=EMBEDDING_DIM   # dimension for embedding
epoch=10         # number of epoch
batch_size=16    # number of examples in batch
lr=0.001         # initial learning rate for adam
schedule=20         # number of epochs to half learning rate
resume=1            # number of epochs to half learning rate
resume_pre_model=0  # resume from pre-training
freeze_encoder_decoder=1      # freeze encoder/decoder weights during training
optimizer='adam'   # optimizer of the model
fine_tune=None    # specific labels id to be fine tuned
inst_norm=1      # use conditional instance normalization in your model
sample_steps=20  # number of batches in between two samples are drawn from validation set
checkpoint_steps=50  # number of batches in between two checkpoints
validate_steps=1     # number of batches in between two validations
validate_batches=20  # validation epochs
flip_labels=1    # whether flip training data labels or not, in fine tuning

In [7]:


with tf.Session(config=config) as sess:
    model = UNet(experiment_dir=experiment_dir, batch_size=batch_size, experiment_id=experiment_id,
                 input_width=image_size, output_width=image_size, embedding_num=embedding_num,
                 validate_batches=validate_batches, embedding_dim=embedding_dim,
                 L1_penalty=L1_penalty, Lconst_penalty=Lconst_penalty,
                 Ltv_penalty=Ltv_penalty, Lcategory_penalty=Lcategory_penalty)
    model.register_session(sess)
    if flip_labels:
        model.build_model(is_training=True, inst_norm=inst_norm, no_target_source=True)
    else:
        model.build_model(is_training=True, inst_norm=inst_norm)
    fine_tune_list = None
    if fine_tune:
        ids = fine_tune.split(",")
        fine_tune_list = set([int(i) for i in ids])
#         print("@@@@@"+str(fine_tune_list))

    model.train(lr=lr, epoch=epoch, resume=resume, resume_pre_model=resume_pre_model,
                schedule=schedule, freeze_encoder_decoder=freeze_encoder_decoder,
                fine_tune=fine_tune_list,
                sample_steps=sample_steps, checkpoint_steps=checkpoint_steps,
                validate_steps=validate_steps,
                flip_labels=flip_labels, optimizer=optimizer)


freeze encoder/decoder weights
unpickled total 86 examples
unpickled total 10 examples
train examples -> 86, val examples -> 10

None

fail to restore model experiments_finetune\checkpoint\experiment_1
fail to restore model experiments_finetune\checkpoint\experiment_1
resume the pre-trained model.....
